In [8]:
import os
import torch
import pandas as pd
from PIL import Image
from torchvision import transforms
from timm import create_model
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# Configs
TEST_IMG_DIR = "/kaggle/input/soil-classification-part-2/soil_competition-2025/test"
MODEL_PATH = "/kaggle/working/convnext_soil_classifier.pth"
BATCH_SIZE = 32
IMG_SIZE = 224
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transforms (same as validation)
test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Dataset for test
class TestDataset(Dataset):
    def __init__(self, img_dir, transform):
        self.img_dir = img_dir
        self.transform = transform
        self.image_ids = sorted(os.listdir(img_dir))

    def __len__(self): return len(self.image_ids)

    def __getitem__(self, idx):
        img_name = self.image_ids[idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)
        return image, img_name

# Load test data
test_dataset = TestDataset(TEST_IMG_DIR, test_transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Load model
model = create_model("convnext_base", pretrained=False, num_classes=1)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()

# Predict
results = []
with torch.no_grad():
    for images, filenames in tqdm(test_loader, desc="Predicting"):
        images = images.to(DEVICE)
        outputs = torch.sigmoid(model(images)).squeeze(1)
        preds = (outputs > 0.5).long().cpu().numpy()
        results.extend(zip(filenames, preds))

# Create submission
submission_df = pd.DataFrame(results, columns=["image_id", "label"])
submission_df.to_csv("submission.csv", index=False)
print("Saved predictions to submission.csv")


Predicting: 100%|██████████| 31/31 [00:13<00:00,  2.36it/s]

Saved predictions to submission.csv
